In [1]:
import pandas as pd
import numpy as np

### 1. 목적
- 네이버 영화리뷰 데이터셋을 이용해서 긍정/부정 분류 모델을 만들어보자
- TF-IDF를 적용해보자

### 2. 데이터 수집
- 네이버 영화 리뷰 데이터 사용

In [2]:
df = pd.read_csv('ratings.txt', delimiter='\t')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        200000 non-null  int64 
 1   document  199992 non-null  object
 2   label     200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [4]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199992 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        199992 non-null  int64 
 1   document  199992 non-null  object
 2   label     199992 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 6.1+ MB


In [5]:
x = df['document']
y = df['label']

In [6]:
x

0                                       어릴때보고 지금다시봐도 재밌어요ㅋㅋ
1         디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...
2                      폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.
3         와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...
4                               안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.
                                ...                        
199995                                       포켓 몬스터 짜가 ㅡㅡ;;
199996                                                쓰.레.기
199997                    완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.
199998                  왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ
199999                                      포풍저그가나가신다영차영차영차
Name: document, Length: 199992, dtype: object

In [7]:
y

0         1
1         1
2         1
3         1
4         1
         ..
199995    0
199996    0
199997    0
199998    0
199999    0
Name: label, Length: 199992, dtype: int64

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   test_size=0.3,
                                                   random_state=0)

In [9]:
x_train

66599                                      선리플 후감상10자압박뒤집어져
38068                                  ㅋㅋ 넘재밌어 나쁜아이없다~~~~~~
96045                 1편보다 크게 못할것까진 없었는데..3편은 더 빵 터지길 기대해봄~
40100                                                 깜동 ..
161918                  대단하다. 어떻게 유명영화를 그대로 베껴서 만드냐ㅋㅋ 중국짱짱맨
                                ...                        
176970                          엄청지루....그냥 그저 그렇게 흘러가다 끝나네요
117956                                   배우들의 카리스마가 너무 부족하다
173692    도저히 몰입할 수없는 캐릭터 설득력없는 스토리 쓸데없는 복선과 배경설정 지나치게 가...
43567                                         영상미는 정말 ㅋㅋㅋㅋㅋ
199348                  시간 아까움1시간이상을 1명에 좁은 공간이라 보는 내가 답답함.
Name: document, Length: 139994, dtype: object

In [10]:
from konlpy.tag import Okt
okt = Okt()

In [11]:
# 명사 토큰화
okt.nouns('오늘은 학교에 가는 날이에요.')

['오늘', '학교', '날']

In [15]:
# 입력 문장이 들어왔을 때 명사만 토큰화 시키는 함수를 선언
def Tokenizer(text) :
    return okt.nouns(text)

In [16]:
# TF-IDF 방식 활용
# tokenizer : 토큰 단위를 지정, 디폴트는 word
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer = Tokenizer)

# 3개 행에 대해서 진행해봄
tfidf.fit(x_train[:3])
tfidf.vocabulary_

{'선': 5,
 '리플': 3,
 '후': 10,
 '감상': 0,
 '자': 7,
 '압박': 6,
 '편': 9,
 '크게': 8,
 '더': 2,
 '빵': 4,
 '기대': 1}

### pos tagging
- 문장을 토큰화 한 후 쪼개진 형태소에 품사를 부여하는 것

In [17]:
# 한글에서는 형용사와 동사에 감정이 많이 들어가있음
data = "오늘 날씨가 추워서 기분이 좋지않다."

In [18]:
okt.morphs(data) # 형태소로 분리

['오늘', '날씨', '가', '추워서', '기분', '이', '좋지', '않다', '.']

In [19]:
okt.pos(data) # 형태소 분리 + 품사 태그

[('오늘', 'Noun'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('추워서', 'Adjective'),
 ('기분', 'Noun'),
 ('이', 'Josa'),
 ('좋지', 'Adjective'),
 ('않다', 'Verb'),
 ('.', 'Punctuation')]

In [20]:
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [17]:
df = pd.DataFrame(okt.pos(data), columns= ['morph','tag'])
df.set_index('tag', inplace=True) # tag를 인덱스로 지정
df.loc[['Verb','Noun','Adjective']]

,morph
tag,
Verb,않다
Noun,오늘
Noun,날씨
Noun,기분
Adjective,추워서
Adjective,좋지


In [21]:
def Tokenizer1(text):
    # 텍스트가 들어오면 데이터프레임으로 변경 후 품사를 인덱스로 설정
    df = pd.DataFrame(okt.pos(text), columns=['morph','tag'])
    df.set_index('tag', inplace=True)
    
    # 동사 명사 형용사일 때 그 값을 반환
    if ('Verb' in df.index)|('Adjective' in df.index) | ('Noun' in df.index):
        labels = ['Verb','Adjective','Noun']
        # intersection() : 괄호 안의 라벨들의 교집합까지 포함해서 배열로 변환
        return df.loc[df.index.intersection(labels)]['morph'].values
    # 형용사 동사 명사 아니면 빈 배열로 반환
    else:
        return []

In [23]:
Tokenizer1(data)

array(['오늘', '날씨', '기분', '추워서', '좋지', '않다'], dtype=object)

In [25]:
tfidf = TfidfVectorizer(tokenizer=Tokenizer1)
tfidf.fit(x_train[:3])
tfidf.vocabulary_ # 동사, 형용사, 명사 추출한 단어모음

/Users/minzy/miniforge3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


{'선': 8,
 '리플': 6,
 '후': 19,
 '감상': 0,
 '자': 12,
 '압박': 9,
 '뒤집어져': 5,
 '넘': 3,
 '재밌어': 13,
 '나쁜아이': 2,
 '없다': 10,
 '편': 16,
 '크게': 14,
 '더': 4,
 '빵': 7,
 '기대': 1,
 '할것까진': 17,
 '터지길': 15,
 '해봄': 18,
 '없었는데': 11}

In [26]:
len(tfidf.vocabulary_)

20

In [31]:
tfidf1 = TfidfVectorizer(tokenizer=Tokenizer1)
tfidf1.fit(x_train)

/Users/minzy/miniforge3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function Tokenizer1 at 0x2a1f4f040>)

In [34]:
# 동사, 형용사, 일반명사만 추출하여 만들어진 단어모음의 개수
len(tfidf1.vocabulary_) 

4188

In [32]:
# 토큰화된 단어를 기계가 이해할 수 있도록 벡터형태로 변환
x_train = tfidf.transform(x_train)
x_test = tfidf.transform(x_test)

### 모델적용

In [35]:
from sklearn.linear_model import LogisticRegression

In [37]:
lr = LogisticRegression()
lr.fit(x_train, y_train)
lr.score(x_test, y_test)

0.529